In [1]:
train_url = 'https://github.com/sidt-ai/MH-hackathons/blob/main/dare_in_reality_2021/data/raw/train.csv?raw=true'
test_url = 'https://raw.githubusercontent.com/sidt-ai/MH-hackathons/main/dare_in_reality_2021/data/raw/test.csv'
# train_weather_url = 'https://raw.githubusercontent.com/sidt-ai/MH-hackathons/main/dare_in_reality_2021/data/raw/train_weather.csv'
# test_weather_url = 'https://raw.githubusercontent.com/sidt-ai/MH-hackathons/main/dare_in_reality_2021/data/raw/test_weather.csv'

In [2]:
import pandas as pd
import numpy as np
import re

In [3]:
train = pd.read_csv(train_url, skipinitialspace=True)
test = pd.read_csv(test_url, skipinitialspace=True)
# train_weather = pd.read_csv(train_weather_url, skipinitialspace=True)
# test_weather = pd.read_csv(test_weather_url, skipinitialspace=True)

In [6]:
test.head()

,NUMBER,DRIVER_NUMBER,LAP_NUMBER,LAP_TIME,LAP_IMPROVEMENT,CROSSING_FINISH_LINE_IN_PIT,S1,S1_IMPROVEMENT,S2,S2_IMPROVEMENT,S3,S3_IMPROVEMENT,KPH,ELAPSED,HOUR,S1_LARGE,S2_LARGE,S3_LARGE,DRIVER_NAME,PIT_TIME,GROUP,TEAM,POWER,LOCATION,EVENT
0,10,1,1,NaN,0,NaN,01:10.2,0,35.433,0,30.693,0,59.5,02:16.4,02:16.4,01:10.2,00:35.4,00:30.7,SB,00:47.0,1.0,JR,NaN,Location 7,Qualifying Group 1
1,10,1,2,NaN,0,NaN,27.667,0,33.68,0,37.767,0,81.8,03:55.5,03:55.5,00:27.7,00:33.7,00:37.8,SB,NaN,1.0,JR,250.0,Location 7,Qualifying Group 1
2,10,1,3,NaN,2,NaN,24.688,2,29.309,2,27.111,2,100.0,05:16.6,05:16.6,00:24.7,00:29.3,00:27.1,SB,NaN,1.0,JR,250.0,Location 7,Qualifying Group 1
3,13,1,1,NaN,0,NaN,01:09.1,0,35.705,0,30.451,0,59.9,02:15.2,02:15.2,01:09.1,00:35.7,00:30.5,ACO,00:44.7,1.0,DSo,NaN,Location 7,Qualifying Group 1
4,13,1,2,NaN,0,NaN,28.401,0,33.693,0,35.958,0,82.7,03:53.3,03:53.3,00:28.4,00:33.7,00:36.0,ACO,NaN,1.0,DSo,250.0,Location 7,Qualifying Group 1


In [14]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 420 entries, 0 to 419
Data columns (total 25 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   NUMBER                       420 non-null    int64  
 1   DRIVER_NUMBER                420 non-null    int64  
 2   LAP_NUMBER                   420 non-null    int64  
 3   LAP_TIME                     0 non-null      float64
 4   LAP_IMPROVEMENT              420 non-null    int64  
 5   CROSSING_FINISH_LINE_IN_PIT  16 non-null     object 
 6   S1                           420 non-null    object 
 7   S1_IMPROVEMENT               420 non-null    int64  
 8   S2                           420 non-null    object 
 9   S2_IMPROVEMENT               420 non-null    int64  
 10  S3                           418 non-null    object 
 11  S3_IMPROVEMENT               420 non-null    int64  
 12  KPH                          418 non-null    float64
 13  ELAPSED             

In [33]:
test.loc[test.S3_LARGE.isna()]

,NUMBER,DRIVER_NUMBER,LAP_NUMBER,LAP_TIME,LAP_IMPROVEMENT,CROSSING_FINISH_LINE_IN_PIT,S1,S1_IMPROVEMENT,S2,S2_IMPROVEMENT,S3,S3_IMPROVEMENT,KPH,ELAPSED,HOUR,S1_LARGE,S2_LARGE,S3_LARGE,DRIVER_NAME,PIT_TIME,GROUP,TEAM,POWER,LOCATION,EVENT
80,25,1,3,NaN,0,B,01:12.1,0,30.11,2,NaN,0,NaN,05:47.7,0:00,01:12.1,00:30.1,NaN,JVER,NaN,1.0,DS¬â,NaN,Location 6,Qualifying Group 1
144,99,1,4,NaN,0,B,27.293,0,33.044,0,NaN,0,NaN,04:58.0,0:00,00:27.3,00:33.0,NaN,PWEHRL,NaN,3.0,TAG,NaN,Location 6,Qualifying Group 3


In [41]:
train.loc[~train.S3_LARGE.isna() &
          (train.LOCATION == 'Location 6') &
          (train.DRIVER_NAME == 'PWEHRL')]['S3_LARGE']

2708    00:26.0
2709    00:21.5
2710    00:20.4
2711    00:19.9
2712    03:54.4
         ...   
8425    00:19.6
8426    00:19.6
8427    00:21.1
8428    00:19.6
8429    00:27.8
Name: S3_LARGE, Length: 62, dtype: object

In [45]:
test.fillna({'PIT_TIME': '00.00.0', 'S3_LARGE': '00:20.0'}, inplace=True)

In [46]:
def to_seconds(t):
  s = str(t)
  parts = list(map(int, re.split('\:|\.', s)))
  return parts[0] * 60 + parts[1] + parts[2] / 10

In [48]:
dummy = pd.DataFrame()
dummy['S1'] = test.S1_LARGE.apply(to_seconds)
dummy['S2'] = test.S2_LARGE.apply(to_seconds)
dummy['S3'] = test.S3_LARGE.apply(to_seconds)
dummy['PIT_TIME'] = test.PIT_TIME.apply(to_seconds)

In [53]:
dummy_submission = pd.DataFrame()
dummy_submission['LAP_TIME'] = dummy.S1 + dummy.S2 + dummy.S3

In [54]:
dummy_submission.head()

,LAP_TIME
0,136.3
1,99.2
2,81.1
3,135.3
4,98.1


In [55]:
dummy_submission.to_csv('dummy_submission.csv', index=False)